<a href="https://colab.research.google.com/github/dayana95/RSO-para-VRP/blob/main/Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install ortools
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

  Using cached ortools-9.10.4067-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached protobuf-5.27.1-cp38-abi3-manylinux2014_x86_64.whl (309 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 whic

In [5]:
#!/usr/bin/env python
# coding: utf-8

# In[25]:


import numpy as np
# Datos del problema
num_locations = 11
distance_matrix = np.array([
[0,  5,  10, 15, 7,  12, 20, 8,  14, 10, 9],
[5,  0,  6,  9,  5,  8,  14, 4,  7,  5,  6],
[10, 6,  0,  8,  4,  6,  13, 7,  5,  8,  9],
[15, 9,  8,  0,  7,  4,  10, 5,  3,  6,  12],
[7,  5,  4,  7,  0,  3,  9,  6,  4,  5,  8],
[12, 8,  6,  4,  3,  0,  7,  3,  5,  6,  9],
[20, 14, 13, 10, 9,  7,  0,  6,  5,  8,  10],
[8,  4,  7,  5,  6,  3,  6,  0,  2,  5,  7],
[14, 7,  5,  3,  4,  5,  5,  2,  0,  3,  8],
[10, 5,  8,  6,  5,  6,  8,  5,  3,  0,  4],
[9,  6,  9,  12, 8,  9,  10, 7,  8,  4,  0]
])

# Crear el gestor de datos
class DataModel:
    def __init__(self):
        self.distance_matrix = distance_matrix
        self.num_vehicles = 1
        self.depot = 0

data = DataModel()

# Crear el gestor del enrutamiento
manager = pywrapcp.RoutingIndexManager(len(data.distance_matrix), data.num_vehicles, data.depot)

# Crear el modelo de enrutamiento
routing = pywrapcp.RoutingModel(manager)

# Función de costo: distancia entre dos ubicaciones
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data.distance_matrix[from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Restricción: cada cliente debe ser visitado exactamente una vez
routing.AddDimension(
    transit_callback_index,
    0,  # sin holgura
    3000,  # distancia máxima permitida
    True,  # el viaje empieza desde el depósito
    'Distance'
)

# Resolver el problema
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

solution = routing.SolveWithParameters(search_parameters)

# Imprimir la solución
if solution:
    index = routing.Start(0)
    plan_output = ''
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    print('Distancia de la ruta: [costo]: {}'.format(route_distance))
else:
    print('No se encontró solución para el problema.')


# In[ ]:



 0 -> 1 -> 2 -> 4 -> 5 -> 3 -> 8 -> 6 -> 7 -> 9 -> 10 -> 0

Distancia de la ruta: [costo]: 54
